# 2. Data description
---

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
crimes_df = pd.read_csv("MontgomeryCountyCrime2013.csv")
crimes_df.head(5)

,Incident ID,CR Number,Dispatch Date / Time,Class,Class Description,Police District Name,Block Address,City,State,Zip Code,...,Sector,Beat,PRA,Start Date / Time,End Date / Time,Latitude,Longitude,Police District Number,Location,Address Number
0,200939101,13047006,10/02/2013 07:52:41 PM,511,BURG FORCE-RES/NIGHT,OTHER,25700 MT RADNOR DR,DAMASCUS,MD,20872.0,...,NaN,NaN,NaN,10/02/2013 07:52:00 PM,NaN,NaN,NaN,OTHER,NaN,25700.0
1,200952042,13062965,12/31/2013 09:46:58 PM,1834,CDS-POSS MARIJUANA/HASHISH,GERMANTOWN,GUNNERS BRANCH RD,GERMANTOWN,MD,20874.0,...,M,5M1,470.0,12/31/2013 09:46:00 PM,NaN,NaN,NaN,5D,NaN,NaN
2,200926636,13031483,07/06/2013 09:06:24 AM,1412,VANDALISM-MOTOR VEHICLE,MONTGOMERY VILLAGE,OLDE TOWNE AVE,GAITHERSBURG,MD,20877.0,...,P,6P3,431.0,07/06/2013 09:06:00 AM,NaN,NaN,NaN,6D,NaN,NaN
3,200929538,13035288,07/28/2013 09:13:15 PM,2752,FUGITIVE FROM JUSTICE(OUT OF STATE),BETHESDA,BEACH DR,CHEVY CHASE,MD,20815.0,...,D,2D1,11.0,07/28/2013 09:13:00 PM,NaN,NaN,NaN,2D,NaN,NaN
4,200930689,13036876,08/06/2013 05:16:17 PM,2812,DRIVING UNDER THE INFLUENCE,BETHESDA,BEACH DR,SILVER SPRING,MD,20815.0,...,D,2D3,178.0,08/06/2013 05:16:00 PM,NaN,NaN,NaN,2D,NaN,NaN


In [3]:
crimes_df.dtypes

Incident ID                 int64
CR Number                   int64
Dispatch Date / Time       object
Class                       int64
Class Description          object
Police District Name       object
Block Address              object
City                       object
State                      object
Zip Code                  float64
Agency                     object
Place                      object
Sector                     object
Beat                       object
PRA                       float64
Start Date / Time          object
End Date / Time            object
Latitude                  float64
Longitude                 float64
Police District Number     object
Location                   object
Address Number            float64
dtype: object

In [4]:
# Null cells per column
crimes_df.isnull().sum()

Incident ID                   0
CR Number                     0
Dispatch Date / Time          0
Class                         0
Class Description             0
Police District Name          0
Block Address                 0
City                          0
State                         0
Zip Code                     30
Agency                        0
Place                         0
Sector                       46
Beat                          8
PRA                           6
Start Date / Time             0
End Date / Time           10178
Latitude                    161
Longitude                   161
Police District Number        0
Location                    161
Address Number              132
dtype: int64

## Columns information
---

| Columns name           | Descriprition | Actual type | Expected type |
| :--------------------- | :------------ | ----------- | ------------- |
| Incident ID            | Unique identifier from database | int64 | - |
| CR Number              | Police Report Number | int64 | - |
| Dispatch Date / Time   | The actual date and time a Officer was dispatched | object | datetime |
| Class                  | Four digit code identifying the crime type of the incident | int64  | - |
| Class Description      | Common name description of the incident class type | object | - |
| Police District Name   | Name of District (Rockville,Weaton etc.) | object | - |
| Block Address          | Address in 100 block level | object | - |
| City                   | City | object | - |
| State                  | State | object | - |
| Zip Code               | Zip Code | float64 | int64 |
| Agency                 | Assigned Police Department | object | - |
| Place                  | Place description | object | - |
| Sector                 | Police Sector Name | object | - |
| Beat                   | Police patrol area subset within District | object | - |
| PRA                    | Police patrol are subset within Beat | float64 | int64 |
| Start Date / Time      | Occurred from date/time | object | datetime |
| End Date / Time        | Occurred to date/time | object | datetime |
| Latitude               | Latitude of the location of the ocorrence | float64 | - |
| Longitude              | Longitude of the location of the ocorrence | float64 | - |
| Police District Number | Major Police Boundary | object | - |
| Location               | Location - This column store the data of the GPS coordinates of the incident as a string in the pattern: (&lt;latitude&gt;, &lt;longitude&gt;) | object | - |
| Address Number         | Address Number - The number extraced from block address column, as same rows does not have a number in 'Block Address', same 'Address Number' have a missing value.| float64 | int64/object 

In [5]:
# Função para comparação entre as colunas de 'Location' e 'Latitude' e 'Longitude'
def check_coordinates(row):
    if type(row['Location']) != float:
        loc_splited = pd.to_numeric(row['Location'][1:-1].split(','))
        return row['Latitude'] == loc_splited[0] and row['Longitude'] == loc_splited[1]
    else:
        return pd.isnull(row['Latitude']) and pd.isnull(row['Longitude'])

(~crimes_df.apply(check_coordinates, axis=1)).sum()

0

In [6]:
# Função para comparação entre as colunas de 'Block Address' e 'Address Number'
def check_addr_number(row):
    block = row['Block Address']
    numbers_in_block = [pd.to_numeric(s) for s in block.split() if s.isdigit()]
    if len(numbers_in_block) > 0: 
        addr_number = pd.to_numeric(numbers_in_block[0])
        # Same number at 'Address Number' and 'Block Address' columns
        return addr_number == row['Address Number']
    else:
        # When 'Block Address' has no number 'Address Number' is null.
        return pd.isnull(row['Address Number'])    
    
(~crimes_df.apply(check_addr_number, axis=1)).sum()

0

## Missing values
---
Almost all columns that have a missing value are associated with localization but the "End Date / Time" which have nearly 40% with 'nan' values. The localization columns with missing values are:
- Zip Code
- Sector
- Beat
- PRA
- Latitude
- Longitude
- Location
- Address Number

Is important to note that e the location column is a pair of latitude and longitude, so the latitude and longitude columns have de same information extracted from location, this implies that the lines with missing values for locatation also missing values for latitude and longitude.

Another derivated column is 'Address Number', which is the first number extracted from the 'Block Address' column, when it have a number. 'Block Address' does not have missing values but 'Address Number' have because same row does not have a number at 'Block Address' even having a address.

This makes the missing values analysis focus on:
- Zip Code
- Sector
- Beat
- PRA

Besides de 'Zip Code', all columns are related with police sectorization. Probabily, these values are missing for not respecting the 100 block level. There still other columns related with police sectorization without missing values:
- Police Disctrict Name
- Agency

These columns also have greater area than the columns with missing values.

# 3. Analyzing the times of crimes
---

In [7]:
dispatch_time = pd.to_datetime(crimes_df['Dispatch Date / Time'])

In [8]:
dispatch_time.dt.weekday_name.value_counts()

Tuesday      3836
Monday       3734
Wednesday    3611
Friday       3594
Thursday     3404
Saturday     2807
Sunday       2383
Name: Dispatch Date / Time, dtype: int64

In [9]:
dispatch_time.dt.weekday.value_counts().sort_index()

0    3734
1    3836
2    3611
3    3404
4    3594
5    2807
6    2383
Name: Dispatch Date / Time, dtype: int64

In [10]:
dispatch_time.dt.hour.value_counts()

7     1278
9     1222
16    1211
15    1179
8     1174
14    1142
13    1132
18    1119
10    1116
17    1115
11    1105
6     1076
20    1065
12    1063
23    1039
19    1030
22    1022
21    1012
0      904
1      855
2      684
3      376
4      228
5      222
Name: Dispatch Date / Time, dtype: int64

In [11]:
dispatch_time.dt.hour.value_counts().sort_index()

0      904
1      855
2      684
3      376
4      228
5      222
6     1076
7     1278
8     1174
9     1222
10    1116
11    1105
12    1063
13    1132
14    1142
15    1179
16    1211
17    1115
18    1119
19    1030
20    1065
21    1012
22    1022
23    1039
Name: Dispatch Date / Time, dtype: int64

In [12]:
dispatch_time.dt.strftime('%B').value_counts()

October      4075
August       4002
November     3941
September    3927
December     3904
July         3520
Name: Dispatch Date / Time, dtype: int64

In [13]:
dispatch_time.dt.month.value_counts().sort_index()

7     3520
8     4002
9     3927
10    4075
11    3941
12    3904
Name: Dispatch Date / Time, dtype: int64

### What day of the week are the most crimes committed on? (ie Monday, Tuesday, etc)

It's simple to see that tuesday is the day most crimes are committed but the diferrence between other days like monday are not much significant.The crimes are more frequent during de week having almost 600 hundreds of diference between saturday and thursday, the weekend day with more crimes and the not weekend day with less crimes.

### During what time of day are the most crimes committed?

Again it's simple to find that the majority of crimes ocur at 7 in the morning but again the difference between other hours are not much significant. But we can draw more info of these date, for example, between 0 hour and 5 hour have a decreasing pattern and the window at 3-5 hours are really low in contrast with the rest of the day. Another interesting aspect is that the peaks are at high intense traffic (7, 9 and 16), hours that people goes to and back school and work.

### During what month are the most crimes committed?

October is the month with more ocurrences of crimes in this dataset, but the crimes occur alike. The most off month is July. The dataset is not extent enough for extract info about that for two reasons: the dataset have only half months of the year of 2013 and have data about only one year. To extract this kind of information of seasonality it's needed more data dispersaded through the hole year.


# 4. Analyzing locations of crimes
---

In [14]:
crimes_df['City'].value_counts()

SILVER SPRING         8626
ROCKVILLE             3453
GAITHERSBURG          3403
GERMANTOWN            2170
BETHESDA              1736
MONTGOMERY VILLAGE     687
POTOMAC                527
CHEVY CHASE            498
OLNEY                  380
KENSINGTON             363
BURTONSVILLE           304
DERWOOD                270
DAMASCUS               230
CLARKSBURG             173
TAKOMA PARK            141
POOLESVILLE            105
BOYDS                   90
BROOKEVILLE             70
SANDY SPRING            43
DICKERSON               26
ASHTON                  19
CABIN JOHN              18
SPENCERVILLE             9
WASHINGTON GROVE         6
BRINKLOW                 5
BARNESVILLE              4
GLEN ECHO                4
MOUNT AIRY               3
LAUREL                   2
BEALLSVILLE              2
HYATTSVILLE              1
KISSIMMEE                1
Name: City, dtype: int64

In [15]:
crimes_df['Police District Name'].value_counts()

SILVER SPRING         5533
WHEATON               4375
MONTGOMERY VILLAGE    3812
ROCKVILLE             3480
BETHESDA              3383
GERMANTOWN            2755
TAKOMA PARK             23
OTHER                    8
Name: Police District Name, dtype: int64

In [16]:
crimes_df['Police District Number'].value_counts()

3D       5533
4D       4375
6D       3812
1D       3480
2D       3383
5D       2755
TPPD       23
OTHER       8
Name: Police District Number, dtype: int64

In [17]:
census_df = pd.read_csv('sub-est2013_24.csv')
pop_by_city_df = census_df[census_df['COUNTY'] == 31][['NAME', 'POPESTIMATE2013']][1:-1]
pop_by_city_df.sort_values('NAME')

,NAME,POPESTIMATE2013
262,Barnesville town,177
263,Brookeville town,138
265,Chevy Chase Section Five village,680
266,Chevy Chase Section Three village,785
267,Chevy Chase View town,955
268,Chevy Chase Village town,2023
264,Chevy Chase town,2918
269,Gaithersburg city,65690
270,Garrett Park town,1030
271,Glen Echo town,262


In [18]:
set(crimes_df['City'].sort_values())

{'ASHTON',
 'BARNESVILLE',
 'BEALLSVILLE',
 'BETHESDA',
 'BOYDS',
 'BRINKLOW',
 'BROOKEVILLE',
 'BURTONSVILLE',
 'CABIN JOHN',
 'CHEVY CHASE',
 'CLARKSBURG',
 'DAMASCUS',
 'DERWOOD',
 'DICKERSON',
 'GAITHERSBURG',
 'GERMANTOWN',
 'GLEN ECHO',
 'HYATTSVILLE',
 'KENSINGTON',
 'KISSIMMEE',
 'LAUREL',
 'MONTGOMERY VILLAGE',
 'MOUNT AIRY',
 'OLNEY',
 'POOLESVILLE',
 'POTOMAC',
 'ROCKVILLE',
 'SANDY SPRING',
 'SILVER SPRING',
 'SPENCERVILLE',
 'TAKOMA PARK',
 'WASHINGTON GROVE'}

### In what area did the most crimes occur? What physical locations (like cities) does this area correspond to?

Analyzing by city we can say that Silver Spring has the most crimes by far, while analizing by Police District, the distribution is not so accetuate at one place but having Silver Spring as the Police District with more crimes. This can be confusing but the city of Silver Spring is located in the border of different police districts which causes de Silver Spring district have different total crimes than the city of Silver Spring.

### Which area has the highest number of crimes per capita?
// Later


# 5. Analyzing types of crime
---

In [19]:
crime_count = crimes_df['Class Description'].value_counts()
crime_count

DRIVING UNDER THE INFLUENCE               1710
CDS-POSS MARIJUANA/HASHISH                1334
POL INFORMATION                           1191
LARCENY FROM AUTO OVER $200                914
LARCENY FROM BUILDING OVER $200            895
MENTAL TRANSPORT                           841
VANDALISM-MOTOR VEHICLE                    762
LOST PROPERTY                              729
LARCENY OTHER OVER $200                    652
LARCENY FROM AUTO UNDER $50                610
FORGERY/CNTRFT - IDENTITY THEFT            596
LARCENY SHOPLIFTING OVER $200              529
LARCENY SHOPLIFTING $50 - $199             489
LIQUOR - DRINK IN PUB OVER 21              482
DISORDERLY CONDUCT                         464
ASSAULT & BATTERY - CITIZEN                382
LARCENY FROM AUTO $50 - $199               356
FORGERY/CNTRFT-CRDT CARDS                  342
MISSING PERSON                             341
SIMPLE ASSAULT - CITIZEN                   311
CDS IMPLMNT-MARIJUANA/HASHISH              297
AUTO THEFT - 

In [20]:
crime_count[crime_count == 1]

BURG NO FORCE - SCH/TIME UNK              1
BURG FORCE - ATTEMPT - SCH/NIGHT          1
ROB KNIFE/CUT - CONV. STORE               1
SUICIDE-ATTEMPT-FIREARM                   1
BURG FORCE-SCH/DAY                        1
VANDALISM GRAFFITI CHURCH/TEMP            1
AGG ASSLT OTHER WPN ON ELDERLY            1
ANIMAL NUISANCE/BARKING                   1
AGG ASSLT FIREARM P.O.                    1
BURG FORCE-SCH/TIME UNK                   1
PARKING OFFENSES                          1
LARCENY COIN MACH OVER $200               1
AGG ASSLT BEAT/INJ ELDERLY                1
EMBEZZLE UNDER $300                       1
SUDDEN DEATH DROWNING                     1
CDS-USE SYNTH DEMEROL/METADONE            1
CDS RX FORGERY INHALANT/GLUE/AEROS        1
TRAFFIC HAZARD                            1
CDS IMPLMNT-BARBITUR/AMPHETAMI            1
ROB OTHER WEAPON GAS/SVC  STA             1
LARCENY COIN MACH $50-$199                1
VANDALISM POSSESSION GRAFFITI MATERIAL    1
HOMICIDE-OTHER                  

In [21]:
len(crime_count[crime_count == 1])

31

In [22]:
set(crimes_df['Class Description'])

{'ABANDONED AUTO',
 'AGG  ASSLT BEAT/INJ P.O.',
 'AGG  ASSLT BEAT/INJ SPOUSE/PARTNER',
 'AGG  ASSLT CUT/STAB OTHR DOMESTC',
 'AGG  ASSLT CUT/STAB P.O.',
 'AGG  ASSLT FIREARM OTHR DOMESTIC',
 'AGG ASSLT BEAT/INJ CTZN',
 'AGG ASSLT BEAT/INJ ELDERLY',
 'AGG ASSLT BEAT/INJ OTHR DOMES',
 'AGG ASSLT CUT/STAB CITIZEN',
 'AGG ASSLT CUT/STAB SPOUSE/PARTNER',
 'AGG ASSLT FIREARM CITIZEN',
 'AGG ASSLT FIREARM P.O.',
 'AGG ASSLT OTHER WPN CITIZEN',
 'AGG ASSLT OTHER WPN ON ELDERLY',
 'AGG ASSLT OTHER WPN OTHR DOMEST',
 'AGG ASSLT OTHER WPN P.O.',
 'AGG ASSLT OTHER WPN SPOUSE/PARTNER',
 'ALL OTHER NON-TRAFFIC CRIM OFFENSES',
 'ANIMAL NUISANCE/BARKING',
 'ANIMAL OFFENSE - AGG ANIMAL/BITE',
 'ANIMAL OFFENSE - HOT CAR',
 'ARSON  MOTOR VEHICLE',
 'ARSON - UNOCCUPIED STRUCT/OTH',
 'ARSON ATTEMPT UNOCCUPIED STRUCTURE/OTH PROP',
 'ARSON UNDER INVEST - UNOCCUP/OTHER',
 'ARSON UNDER INVEST - VEHICLE',
 'ARSON- OCCUPIED STRUCTURE',
 'ASSAULT & BATTERY - CITIZEN',
 'ASSAULT & BATTERY - ELDERLY',
 'ASSAULT & B

In [23]:
violent_pattern = ['WEAPON', 'ASSLT', 'ASSAULT', 'KNIFE', 'FIREARM', 'FIRE OTHER', 'HOMICIDE', 'RAPE', ]
crimes_df['Violent'] = crimes_df['Class Description'].str.contains('|'.join(violent_pattern))
crimes_df['Violent']

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13        True
14       False
15       False
16       False
17       False
18       False
19       False
20        True
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
23339    False
23340    False
23341     True
23342    False
23343    False
23344    False
23345    False
23346    False
23347    False
23348    False
23349    False
23350    False
23351    False
23352    False
23353    False
23354    False
23355    False
23356    False
23357    False
23358    False
23359     True
23360    False
23361    False
23362    False
23363    False
23364     True
23365     True
23366     True
23367    False
23368    False
Name: Violent, Length: 23369, dtype: bool

In [24]:
crimes_df[crimes_df['Violent']]['Class Description'].value_counts()

ASSAULT & BATTERY - CITIZEN            382
SIMPLE ASSAULT - CITIZEN               311
ASSAULT & BATTERY SPOUSE/PARTNER       290
SIMPLE ASSAULT SPOUSE/PARTNER          111
ASSAULT & BATTERY OTHER DOMESTIC        88
SIMPLE ASSAULT OTHER DOMESTIC           87
ROB FIREARM - STREET                    56
ASSAULT & BATTERY - POLICE OFFICER      55
SIMPLE ASSAULT - PO                     50
WEAPON CONCEALED OTHER WEAPON           49
SEX OFFENSE - SEX  ASSAULT              47
WEAPON POSSESSION OTHER WEAPON          39
RAPE-FORCE                              36
AGG ASSLT CUT/STAB CITIZEN              36
FIRE OTHER                              34
RECOVERED PROPERTY - FIREARM            30
AGG ASSLT OTHER WPN CITIZEN             26
ROB KNIFE/CUT INST - STREET             24
WEAPON POSSESSION HANDGUN               24
AGG ASSLT BEAT/INJ CTZN                 18
WEAPON CONCEALED HANDGUN                15
AGG  ASSLT BEAT/INJ SPOUSE/PARTNER      15
AGG ASSLT FIREARM CITIZEN               15
AGG ASSLT O

In [25]:
crimes_df[~crimes_df['Violent']]['Class Description'].value_counts()

DRIVING UNDER THE INFLUENCE               1710
CDS-POSS MARIJUANA/HASHISH                1334
POL INFORMATION                           1191
LARCENY FROM AUTO OVER $200                914
LARCENY FROM BUILDING OVER $200            895
MENTAL TRANSPORT                           841
VANDALISM-MOTOR VEHICLE                    762
LOST PROPERTY                              729
LARCENY OTHER OVER $200                    652
LARCENY FROM AUTO UNDER $50                610
FORGERY/CNTRFT - IDENTITY THEFT            596
LARCENY SHOPLIFTING OVER $200              529
LARCENY SHOPLIFTING $50 - $199             489
LIQUOR - DRINK IN PUB OVER 21              482
DISORDERLY CONDUCT                         464
LARCENY FROM AUTO $50 - $199               356
FORGERY/CNTRFT-CRDT CARDS                  342
MISSING PERSON                             341
CDS IMPLMNT-MARIJUANA/HASHISH              297
AUTO THEFT - PASSENGER VEHICLE             293
LARCENY AUTO PART UNDER $50                267
LARCENY SHOPL

### Which crimes are the most common? Least common?

Driving under influence and marijuana/hashish possesion are the most common crimes in the data. We can see the number of crimes of others types rapidly descreses, making these two crimes signifcantly more common. While tied in least common we have 31 types of occurrences. These effect of tiedness can be smoothly by grouping the occurrences by some common feature.

### Can you split the types of crimes manually into "Violent" (caused harm to others or involved weapons) and "Nonviolent" (mostly property crimes, like theft)? What's the most common violent crime? The most common nonviolent?

To classify the occurrences in violent and non-violent each row was verified to see if contains one of the following expressions:

- WEAPON
- ASSLT
- ASSAULT
- KNIFE
- FIREARM
- FIRE OTHER
- HOMICIDEASSAULT & BATTERY - CITIZEN
- RAPE

Now it's possible to evaluate that the most common violent occurrence is assult and battery on a citizen and the non-violent stills driving under influence.

# 6. Combine Analysis
---

In [26]:
crimes_df['Dispatch Date / Time'] = pd.to_datetime(crimes_df['Dispatch Date / Time'])
crimes_df['Hour'] = crimes_df['Dispatch Date / Time'].dt.hour
most_common_violent_crimes = crimes_df[crimes_df['Violent']]['Class Description'].value_counts().index[0:10].tolist()
most_common_non_violent_crimes = crimes_df[~crimes_df['Violent']]['Class Description'].value_counts().index[0:10].tolist()

mcvc_df = crimes_df[crimes_df['Class Description'].isin(most_common_violent_crimes)]
mcnvc_df = crimes_df[crimes_df['Class Description'].isin(most_common_non_violent_crimes)]

In [27]:
mcvc_df[['City', 'Class Description', 'Incident ID']].groupby(['City', 'Class Description']).count()

Incident ID
City          Class Description                              
ASHTON        ASSAULT & BATTERY OTHER DOMESTIC              1
BETHESDA      ASSAULT & BATTERY - CITIZEN                  25
              ASSAULT & BATTERY - POLICE OFFICER            4
              ASSAULT & BATTERY OTHER DOMESTIC              2
              ASSAULT & BATTERY SPOUSE/PARTNER              8
              ROB FIREARM - STREET                          2
              SIMPLE ASSAULT - CITIZEN                     21
              SIMPLE ASSAULT - PO                           2
              SIMPLE ASSAULT OTHER DOMESTIC                 2
              SIMPLE ASSAULT SPOUSE/PARTNER                 4
              WEAPON CONCEALED OTHER WEAPON                 3
BOYDS         ASSAULT & BATTERY - CITIZEN                   1
              SIMPLE ASSAULT SPOUSE/PARTNER                 2
BROOKEVILLE   ASSAULT & BATTERY - CITIZEN                   1
              ASSAULT & BATTERY SPOUSE/PARTNER              1
              SIMPLE ASSAULT - CITIZEN                      1
BURTONSVILLE  ASSAULT & BATTERY - CITIZEN                   3
              ASSAULT & BATTERY SPOUSE/PARTNER              3
              ROB FIREARM - STREET                          1
              SIMPLE ASSAULT - CITIZEN                      7
              SIMPLE ASSAULT - PO                           1
              SIMPLE ASSAULT OTHER DOMESTIC                 1
CHEVY CHASE   ASSAULT & BATTERY - CITIZEN                   1
              ASSAULT & BATTERY - POLICE OFFICER            2
              ASSAULT & BATTERY SPOUSE/PARTNER              4
              ROB FIREARM - STREET                          1
              SIMPLE ASSAULT - CITIZEN                      3
              SIMPLE ASSAULT - PO                           1
              SIMPLE ASSAULT OTHER DOMESTIC                 1
CLARKSBURG    ASSAULT & BATTERY - CITIZEN                   4
...                                                       ...
POTOMAC       SIMPLE ASSAULT - CITIZEN                      8
              SIMPLE ASSAULT OTHER DOMESTIC                 1
              SIMPLE ASSAULT SPOUSE/PARTNER                 4
ROCKVILLE     ASSAULT & BATTERY - CITIZEN                  46
              ASSAULT & BATTERY - POLICE OFFICER            5
              ASSAULT & BATTERY OTHER DOMESTIC              8
              ASSAULT & BATTERY SPOUSE/PARTNER             41
              ROB FIREARM - STREET                          6
              SIMPLE ASSAULT - CITIZEN                     61
              SIMPLE ASSAULT - PO                           8
              SIMPLE ASSAULT OTHER DOMESTIC                14
              SIMPLE ASSAULT SPOUSE/PARTNER                10
              WEAPON CONCEALED OTHER WEAPON                10
SANDY SPRING  ASSAULT & BATTERY SPOUSE/PARTNER              2
SILVER SPRING ASSAULT & BATTERY - CITIZEN                 157
              ASSAULT & BATTERY - POLICE OFFICER           24
              ASSAULT & BATTERY OTHER DOMESTIC             44
              ASSAULT & BATTERY SPOUSE/PARTNER            124
              ROB FIREARM - STREET                         41
              SIMPLE ASSAULT - CITIZEN                    116
              SIMPLE ASSAULT - PO                          19
              SIMPLE ASSAULT OTHER DOMESTIC                34
              SIMPLE ASSAULT SPOUSE/PARTNER                41
              WEAPON CONCEALED OTHER WEAPON                22
TAKOMA PARK   ASSAULT & BATTERY - CITIZEN                   1
              ASSAULT & BATTERY - POLICE OFFICER            1
              ASSAULT & BATTERY OTHER DOMESTIC              2
              ASSAULT & BATTERY SPOUSE/PARTNER              6
              SIMPLE ASSAULT - PO                           1
              SIMPLE ASSAULT SPOUSE/PARTNER                 2

[129 rows x 1 columns]

In [28]:
mcnvc_df[['City', 'Class Description', 'Incident ID']].groupby(['City', 'Class Description']).count()

Incident ID
City             Class Description                           
ASHTON           CDS-POSS MARIJUANA/HASHISH                 1
                 DRIVING UNDER THE INFLUENCE                2
                 LARCENY FROM AUTO UNDER $50                1
                 LARCENY FROM BUILDING OVER $200            2
                 LARCENY OTHER OVER $200                    1
                 MENTAL TRANSPORT                           1
                 POL INFORMATION                            2
BARNESVILLE      MENTAL TRANSPORT                           1
                 POL INFORMATION                            2
BEALLSVILLE      LARCENY FROM BUILDING OVER $200            1
                 POL INFORMATION                            1
BETHESDA         CDS-POSS MARIJUANA/HASHISH                28
                 DRIVING UNDER THE INFLUENCE              183
                 LARCENY FROM AUTO OVER $200               73
                 LARCENY FROM AUTO UNDER $50               49
                 LARCENY FROM BUILDING OVER $200          133
                 LARCENY OTHER OVER $200                   67
                 LOST PROPERTY                             75
                 MENTAL TRANSPORT                          54
                 POL INFORMATION                           73
                 VANDALISM-MOTOR VEHICLE                   44
BOYDS            CDS-POSS MARIJUANA/HASHISH                 3
                 DRIVING UNDER THE INFLUENCE                4
                 LARCENY FROM AUTO OVER $200                1
                 LARCENY FROM AUTO UNDER $50                2
                 LARCENY FROM BUILDING OVER $200            1
                 LARCENY OTHER OVER $200                    4
                 LOST PROPERTY                              3
                 MENTAL TRANSPORT                           3
                 POL INFORMATION                            8
...                                                       ...
SANDY SPRING     MENTAL TRANSPORT                           2
                 POL INFORMATION                            3
SILVER SPRING    CDS-POSS MARIJUANA/HASHISH               525
                 DRIVING UNDER THE INFLUENCE              561
                 LARCENY FROM AUTO OVER $200              343
                 LARCENY FROM AUTO UNDER $50              209
                 LARCENY FROM BUILDING OVER $200          282
                 LARCENY OTHER OVER $200                  201
                 LOST PROPERTY                            206
                 MENTAL TRANSPORT                         264
                 POL INFORMATION                          377
                 VANDALISM-MOTOR VEHICLE                  279
SPENCERVILLE     DRIVING UNDER THE INFLUENCE                3
                 LARCENY FROM BUILDING OVER $200            1
                 LARCENY OTHER OVER $200                    1
                 MENTAL TRANSPORT                           1
                 POL INFORMATION                            2
TAKOMA PARK      CDS-POSS MARIJUANA/HASHISH                10
                 DRIVING UNDER THE INFLUENCE                4
                 LARCENY FROM AUTO OVER $200                6
                 LARCENY FROM AUTO UNDER $50                3
                 LARCENY FROM BUILDING OVER $200            2
                 LARCENY OTHER OVER $200                    2
                 LOST PROPERTY                              1
                 MENTAL TRANSPORT                           5
                 POL INFORMATION                            8
                 VANDALISM-MOTOR VEHICLE                    6
WASHINGTON GROVE CDS-POSS MARIJUANA/HASHISH                 2
                 DRIVING UNDER THE INFLUENCE                2
                 MENTAL TRANSPORT                           1

[220 rows x 1 columns]

In [29]:
mcvc_df[['Hour', 'Class Description', 'Incident ID']].groupby(['Hour', 'Class Description']).count()

Incident ID
Hour Class Description                              
0    ASSAULT & BATTERY - CITIZEN                  17
     ASSAULT & BATTERY - POLICE OFFICER            6
     ASSAULT & BATTERY OTHER DOMESTIC              9
     ASSAULT & BATTERY SPOUSE/PARTNER             18
     ROB FIREARM - STREET                          6
     SIMPLE ASSAULT - CITIZEN                     10
     SIMPLE ASSAULT - PO                           4
     SIMPLE ASSAULT OTHER DOMESTIC                 9
     SIMPLE ASSAULT SPOUSE/PARTNER                 5
     WEAPON CONCEALED OTHER WEAPON                 2
1    ASSAULT & BATTERY - CITIZEN                  19
     ASSAULT & BATTERY - POLICE OFFICER            1
     ASSAULT & BATTERY OTHER DOMESTIC              6
     ASSAULT & BATTERY SPOUSE/PARTNER             17
     ROB FIREARM - STREET                          4
     SIMPLE ASSAULT - CITIZEN                      5
     SIMPLE ASSAULT - PO                           6
     SIMPLE ASSAULT OTHER DOMESTIC                 3
     SIMPLE ASSAULT SPOUSE/PARTNER                 5
     WEAPON CONCEALED OTHER WEAPON                 2
2    ASSAULT & BATTERY - CITIZEN                  13
     ASSAULT & BATTERY - POLICE OFFICER            7
     ASSAULT & BATTERY OTHER DOMESTIC              2
     ASSAULT & BATTERY SPOUSE/PARTNER             12
     ROB FIREARM - STREET                          3
     SIMPLE ASSAULT - CITIZEN                     13
     SIMPLE ASSAULT - PO                           1
     SIMPLE ASSAULT OTHER DOMESTIC                 6
     SIMPLE ASSAULT SPOUSE/PARTNER                 5
     WEAPON CONCEALED OTHER WEAPON                 4
...                                              ...
21   ASSAULT & BATTERY - CITIZEN                  25
     ASSAULT & BATTERY - POLICE OFFICER            2
     ASSAULT & BATTERY OTHER DOMESTIC              5
     ASSAULT & BATTERY SPOUSE/PARTNER             23
     ROB FIREARM - STREET                          2
     SIMPLE ASSAULT - CITIZEN                     15
     SIMPLE ASSAULT - PO                           4
     SIMPLE ASSAULT OTHER DOMESTIC                11
     SIMPLE ASSAULT SPOUSE/PARTNER                 7
     WEAPON CONCEALED OTHER WEAPON                 3
22   ASSAULT & BATTERY - CITIZEN                  14
     ASSAULT & BATTERY - POLICE OFFICER            7
     ASSAULT & BATTERY OTHER DOMESTIC              2
     ASSAULT & BATTERY SPOUSE/PARTNER             28
     ROB FIREARM - STREET                         10
     SIMPLE ASSAULT - CITIZEN                     21
     SIMPLE ASSAULT - PO                           3
     SIMPLE ASSAULT OTHER DOMESTIC                 2
     SIMPLE ASSAULT SPOUSE/PARTNER                11
     WEAPON CONCEALED OTHER WEAPON                 2
23   ASSAULT & BATTERY - CITIZEN                  29
     ASSAULT & BATTERY - POLICE OFFICER            7
     ASSAULT & BATTERY OTHER DOMESTIC              6
     ASSAULT & BATTERY SPOUSE/PARTNER             17
     ROB FIREARM - STREET                          6
     SIMPLE ASSAULT - CITIZEN                      9
     SIMPLE ASSAULT - PO                           5
     SIMPLE ASSAULT OTHER DOMESTIC                 2
     SIMPLE ASSAULT SPOUSE/PARTNER                 8
     WEAPON CONCEALED OTHER WEAPON                 2

[216 rows x 1 columns]

In [30]:
mcnvc_df[['Hour', 'Class Description', 'Incident ID']].groupby(['Hour', 'Class Description']).count()

Incident ID
Hour Class Description                           
0    CDS-POSS MARIJUANA/HASHISH               135
     DRIVING UNDER THE INFLUENCE              257
     LARCENY FROM AUTO OVER $200                9
     LARCENY FROM AUTO UNDER $50                5
     LARCENY FROM BUILDING OVER $200            7
     LARCENY OTHER OVER $200                    4
     LOST PROPERTY                              4
     MENTAL TRANSPORT                          27
     POL INFORMATION                           33
     VANDALISM-MOTOR VEHICLE                    8
1    CDS-POSS MARIJUANA/HASHISH                91
     DRIVING UNDER THE INFLUENCE              320
     LARCENY FROM AUTO OVER $200                3
     LARCENY FROM AUTO UNDER $50                9
     LARCENY FROM BUILDING OVER $200            5
     LARCENY OTHER OVER $200                    4
     LOST PROPERTY                              3
     MENTAL TRANSPORT                          24
     POL INFORMATION                           22
     VANDALISM-MOTOR VEHICLE                   11
2    CDS-POSS MARIJUANA/HASHISH                41
     DRIVING UNDER THE INFLUENCE              295
     LARCENY FROM AUTO OVER $200                6
     LARCENY FROM AUTO UNDER $50                5
     LARCENY FROM BUILDING OVER $200            2
     LARCENY OTHER OVER $200                    3
     LOST PROPERTY                              3
     MENTAL TRANSPORT                          24
     POL INFORMATION                           17
     VANDALISM-MOTOR VEHICLE                    9
...                                           ...
21   CDS-POSS MARIJUANA/HASHISH               103
     DRIVING UNDER THE INFLUENCE               81
     LARCENY FROM AUTO OVER $200               19
     LARCENY FROM AUTO UNDER $50                6
     LARCENY FROM BUILDING OVER $200           26
     LARCENY OTHER OVER $200                   20
     LOST PROPERTY                              5
     MENTAL TRANSPORT                          51
     POL INFORMATION                           56
     VANDALISM-MOTOR VEHICLE                   32
22   CDS-POSS MARIJUANA/HASHISH               159
     DRIVING UNDER THE INFLUENCE              118
     LARCENY FROM AUTO OVER $200               13
     LARCENY FROM AUTO UNDER $50                5
     LARCENY FROM BUILDING OVER $200           20
     LARCENY OTHER OVER $200                   16
     LOST PROPERTY                             13
     MENTAL TRANSPORT                          37
     POL INFORMATION                           30
     VANDALISM-MOTOR VEHICLE                   16
23   CDS-POSS MARIJUANA/HASHISH               180
     DRIVING UNDER THE INFLUENCE              204
     LARCENY FROM AUTO OVER $200                9
     LARCENY FROM AUTO UNDER $50                1
     LARCENY FROM BUILDING OVER $200            5
     LARCENY OTHER OVER $200                    8
     LOST PROPERTY                              8
     MENTAL TRANSPORT                          38
     POL INFORMATION                           31
     VANDALISM-MOTOR VEHICLE                    9

[239 rows x 1 columns]

### Where are the most violent crimes committed? How about nonviolent?

The first table shows how much occurrences of the top 10 most common violent occurrences at each city. The second table shows how much occurrences of the top 10 most common non violent at each city.

### When are the most violent crimes committed? How about nonviolent?

The third table shows how much occurrences of the top 10 most common violent occurrences per hour of the day. The fourth table shows how much occurrences of the top 10 most common non violent occurrences per hour of the day.

# 7. Posing and answering your own questions
---

We propose to look after information related with location, time and the combination of time and location with the objective to understand patterns and be able to better use resource to prevent and respond to occurences. All the analisys will use a categorization of the occurrence's Class. These analysis will separated in notebooks:
- Classificacao_ocorrencias: for the classifying algorithm
- Location Analysis: for the location analysis
- Time Analysis: for the time analysis
- Combined Analysis: for analysis related to both time and location



# Referências

[Descrição de códigos de classe][response_codes]: Está página contém a descrição das classes das ocorrências listadas no dataset e define um agrupamento para os códigos. Estes agrupamentos foram utilizados para classificar as ocorrências, foi necessário criar alguns agrupamentos para códigos específicos que não possuem grupos no link referenciado.

[Relatório policial Rockville][uniform_crime_stat_report]: Relatório policial comparativo entre os anos de 2013 e 2014. Este relatório foi utilizado para obter algum entendimento sobre o sistema policial de relatórios implementado nos Estados Unidos.

[Manual para sistema de relatório uniforme de crimes][ucr_handbook]: Uniform Crime Reporting é uma iniciativa do governo americano para análise estatística dos crimes. Foi concebido em 1929 e em 1930 o FBI foi incubido de coletar, publicar e armazenar esses dados.

[Mapa policial][mont_county_pol_map]: Mapa Policial interativo do Condado de Montgomenry, Maryland.

[Fonte dataset][dataset]: Link com origem do dataset com definições sobre as colunas, também provê um ambiente para análise.

[response_codes]: http://wiki.radioreference.com/index.php/Montgomery_County_(MD)_Response_Codes "Montgomery County (MD) Response Codes"
[uniform_crime_stat_report]: http://www.rockvillemd.gov/DocumentCenter/View/10969 "Uniform Crime Statistics Report - Rockville City Police Department"
[ucr_handbook]: https://www2.fbi.gov/ucr/handbook/ucrhandbook04.pdf "Uniform Crime Report Handbook"
[mont_county_pol_map]: http://mcgov-gis.maps.arcgis.com/apps/Viewer/index.html?appid=4317830a05654b8f907e65515970a5ba "Montgomery County Police Map"

[dataset]: https://data.policefoundation.org/Incidents/Montgomery-County-MD-MCPD-Incidents-07-2003-/c2mn-zwn5 "Montgomery County, MD - MCPD Incidents "